# Exercise 2

Exercise 2 includes a **written assignment** (10 points), a **programming assignment with 6 problems** (9 points) and a **feedback/workload assessment assignment** (1 point). For each problem you need to modify the notebook by adding your own programming solutions or written text. Remember to save and commit your changes locally, and push your changes to GitHub after each major change! Regular commits will help you to keep track of your changes (and revert them if needed). Pushing your work to GitHub will ensure that you don't lose any work in case your computer crashes (can happen!).

### Time allocation

**Completing this exercise takes approximately: 12-16 hours** (based on previous year statistics). However, the time it takes can vary significantly from student to student, so we **recommended that you start immediately working on the exercise to avoid issues meeting with the deadline**.

### Due date

You should submit (push) your Exercise answers to your personal Github repository **two weeks after the first practical session (submit by Sunday 5.2)**. 

### Start your exercise in CSC Notebooks

Before you can start programming, you need to launch the CSC Notebook instance and clone your **personal copy of the Exercise repository** (i.e. something like `exercise-2-htenkanen`) there using Git. If you need help with this, [read the documentation on the course site](https://sustainability-gis.readthedocs.io/en/latest/lessons/L1/git-basics.html).

### Working with Jupyter Notebooks

Jupyter Notebooks are documents that can be used and run inside the JupyterLab programming environment (e.g. at [notebooks.csc.fi](https://notebooks.csc.fi/)) containing the computer code and rich text elements (such as text, figures, tables and links). 

**A couple of hints**:

- You can **execute a cell** by clicking a given cell that you want to run and pressing <kbd>Shift</kbd> + <kbd>Enter</kbd> (or by clicking the "Play" button on top)
- You can **change the cell-type** between `Markdown` (for writing text) and `Code` (for writing/executing code) from the dropdown menu above. 

See [**further details and help from here**](https://pythongis.org/part1/chapter-01/nb/04-using-jupyterlab.html). 
 
### Hints 

If there are general questions arising from this exercise, we will add hints to the course website under [Exercise 2 description](https://sustainability-gis.readthedocs.io/en/latest/lessons/L2/exercise-2.html). 

<hr style="border:2px solid gray">

# Part 1: Written assignment (10 points)

In the *Human wellbeing and capabilities* and *Network analytics and spatial accessibility modelling* lessons this week, we went through concepts related to equity, justice and spatial accessibility and learned how spatial network analysis can be used to quantify access. 


## Equity and accessibility

Write approx. 0.5-2 pages (A4) of text in English. Remember to **cite your sources appropriately** if you use any literature or other reading materials in your text. 

In this essay, you should cover following questions:
 
 - How philosophies of justice and concept of accessibility relate to each other?
   - how justice and equity can be understood (e.g. Rawlsian view)?
 - How spatial accessibility can be modelled and what kind of application areas there exists?
 - How equity and spatial accessibility modelling relates to sustainability?
 - Is equity and justice something that you are thinking regularly? Or at some point in your life? Please share your thoughts.
 
Use the lesson materials and the recommended readings (optional) as a source of information for answering to these.

### Grading criteria for the essay

- Answers to the question(s): 5 points

- Reflection against literature + materials: 3 points

- Fluency / clarity of the text: 1 point

- Appropriate citation practices used: 1 point

----------------

## Answers / reflection

**Add your text here.**

*Hint: To "activate" this cell in Editing mode, double click this cell. If you want to get this cell back in the "Reading-mode", <kbd>Shift</kbd> + <kbd>Enter</kbd>.*

## Hints

- If you need help in Markdown formatting (e.g. how to add headings, bold, italics, links etc.), please take a look at this excellent [guide / cheatsheet](https://www.markdownguide.org/cheat-sheet/) 

<hr style="border:2px solid gray">

# Part 2: Programming assignment

In this exercise, we will practice network analysis and spatial accessibility modelling. Our overall goal is to analyze healthcare accessibility by conducting a two-step floating catchment area analysis in Helsinki Region. We will calculate travel time-based catchment areas by public transport (30 minutes threshold) from healthcare facilities (Health stations, University hospitals and City hospitals) located in the area to  population grid cells (250 meter resolution) in the Helsinki Region. Based on the population information and the number of physicians at given hospital (simulated --> not real), we can calculate an accessibility index based on 2SFCA method. The end result should look something like following:

!["Healthcare accessibility map in Helsinki Region](img/hospital_accessibility_2SFCA_HelsinkiRegion.png)
  
The main point in this Exercise is to practice how to do accessibility analysis in Python, and use it for a practical health-related research question. Similar approaches can be used for many different applications that require understanding supply and demand of some kind of service (can be e.g. commercial application as well).

### Start your exercise in CSC Notebooks

Before you can start programming, you need to launch the CSC Notebook instance and clone your Exercise repository there.
If you need help with this, [read the documentation on the course site](https://sustainability-gis.readthedocs.io/en/latest/lessons/L1/git-basics.html).
 
### Hints 

If there are general questions arising from this exercise, we will add hints to the course website under [Exercise 2 description](https://sustainability-gis.readthedocs.io/en/latest/lessons/L2/exercise-2.html). 

## Input data for the exercise

We will use only openly available data sources for this exercise, listed below.

### Healthcare facilities and number of physicians 

We will use the real locations of healthcare facilities in Helsinki Region (based on data from [Palvelukartta](https://palvelukartta.hel.fi/fi/)) and **simulated** data about the number of doctors working in them. The distribution of the number of doctors working in different facilities looks like this (the size of the bubble corresponds the number of physicians):
![The proportion of doctors in different Helsinki Region health facilities](img/Number_of_doctors_in_hospitals_simulated.png)

Notice that this data **is not real** and it has been simulated based on openly available data/information and has many assumptions (e.g. that the number of doctors in cities reflect the average of whole Finland which is 3.8 doctors per 1000 inhabitants). The physicians are allocated to different facilities based on relative number of health-related employees working at different postal code areas (again a big assumption). These steps have been taken to make the data a bit more realistic, but they cannot be considered to reflect reality. If you are interested to see the details of how the data was produced based on open data, [check this notebook](https://github.com/AaltoGIS/data-preparations/blob/master/Preparing_hospital_data.ipynb). 

### Population distribution

The population data that we will use in this exercise is openly available from [HSY](https://www.hsy.fi/en/environmental-information/open-data/avoin-data---sivut/population-grid-of-helsinki-metropolitan-area/). The data is from 2019 and it has been cleaned, so it is ready for you to use. The column `pop_cnt` contains information about the number of inhabitants in given grid cell (note: the original column name is "ASUKKAITA"). The population distribution in Helsinki looks like following:

![Population distribution](img/Number_of_inhabitants_Helsinki_Region_2019.PNG)

### Street network and GTFS

For conducting the accessibility analysis, we will use OpenStreetMap and GTFS data using `r5py` library.  

## Helper functions

The following helper functions will be used in the exercise for parsing catchment area information for hospitals. **Execute this cell before starting to work on the exercise.**

In [ ]:
def prepare_network(osm_fp, gtfs_fp, memory_in_gigabytes=6):
    """
    Helper function to create a routable network for r5py based on OpenStreetMap data and GTFS data.
    
    Parameters
    ----------
    
    osm_fp : str
    
        Filepath to the OpenStreetMap PBF file (*.osm.pbf).
        
    gtfs_fp : str | list
    
        Filepath to the GTFS zip-file, or alternatively a list of multiple filepaths to GTFS Zipfiles.
        
    memory_in_gigabytes : int
        
        The maximum amount of memory in Gigabytes which can be consumed by r5py. 
    """
    
    # Allocate memory (6 GiB by default)
    import sys
    sys.argv.append(["--max-memory", f"{memory_in_gigabytes}G"])
    from r5py import TransportNetwork

    if isinstance(gtfs_fp, str):
        gtfs_fp = [gtfs_fp]
    elif isinstance(gtfs_fp, list):
        # Check that the inputs are valid
        for item in gtfs_fp:
            assert isinstance(item, str), f"All objects in 'gtfs_fp' list should be filepath strings. Got '{type(item)}'."

    # Build network
    net = TransportNetwork(osm_fp,  gtfs_fp)
    return net

def make_bubble_map(gdf, radius_column, column=None, cmap="hsv", tiles="CartoDB darkmatter"):
    """
    Helper function to generate bubble map from input Point dataset.
    
    Parameters
    ----------
    
    gdf : gpd.GeoDataFrame
        
        The GeoDataFrame that should contain points.
    
    radius_column : str
        
        Name for the column which determines the size of the bubble (required).
    
    column : str
    
        Name for the column which will be used to determine a color of the bubble (optional).
    
    cmap : str
    
        Colormap which will be determining the colors if `column` variable is used.
    
    tiles : str
    
        Background map style. By default, plots using a dark background (CartoDB darkmatter)
    
    
    """
    import numpy as np
    
    # Generate bubble map
    m = gdf.explore(column=column, cmap=cmap,
                    
                    # Generate Bubbles based on given radius column
                    style_kwds={"style_function": lambda x: {
                            "radius": np.sqrt(x["properties"][radius_column])*0.7,
                            "stroke": True,  
                            "color": "gray",
                            "weight": 1}
                            },
                    # Background map style
                    tiles="CartoDB darkmatter",
                   )
    return m

def parse_id_of_the_closest_facility(travel_time_matrix, origin_column="from_id", destination_column="to_id", distance_column="travel_time"):
    """
    A helper function to define catchment areas for given facilities. The function will find out which origin is closest to the given destinations based on travel time. 
    The 'id' of the origins will be stored to a new column called "facility_id". 
    
    Parameters
    ----------
    
    travel_time_matrix : pd.DataFrame
    
        DataFrame containing the data with travel time information.
     
    origin_column : str
    
        The name of the column containing origin ids (typically 'from_id' column).
    
    destination_column : str
    
        The name of the column containing destination ids (typically 'to_id' column).
    
    distance_column : str
    
        The name of the column containing the travel times (or other distance information).
    """
    # Set the origin as index (i.e. in our case the column name of the id representing hospitals in the matrix)
    travel_time_matrix = travel_time_matrix.set_index(origin_column)

    # Find the from_id of the closest facility based on travel time
    closest_facility = travel_time_matrix.groupby(destination_column)[distance_column].idxmin().reset_index()

    # Rename the column from "travel_time" to more intuitive one
    closest_facility = closest_facility.rename(columns={"travel_time": "facility_id"})
    
    return closest_facility

def parse_travel_time_to_the_closest_facility(travel_time_matrix, origin_column="from_id", destination_column="to_id", distance_column="travel_time"):
    """
    A helper function to parse the travel times for closest facilities. The function will find out which origin is closest 
    to the given destinations based on travel time and returns travel times for the closest facilities. 
    
    Parameters
    ----------
    
    travel_time_matrix : pd.DataFrame
    
        DataFrame containing the data with travel time information.
     
    origin_column : str
    
        The name of the column containing origin ids (typically 'from_id' column).
    
    destination_column : str
    
        The name of the column containing destination ids (typically 'to_id' column).
    
    distance_column : str
    
        The name of the column containing the travel times (or other distance information).
    """
    
    # Find the from_id of the closest facility based on travel time
    closest_facility = travel_time_matrix.groupby(destination_column)[distance_column].min().reset_index()

    return closest_facility

def calculate_2SFCA(travel_time_matrix, ppr_gdf, population_grid_gdf):
    """
    A helper function to calculate accessibility index based on Two-step Floating Catchment Area method. 
    
    Parameters
    ----------
    
    travel_time_matrix : pd.DataFrame
    
        DataFrame containing the data with travel time information.
     
    ppr_gdf : gpd.GeoDataFrame
    
        A GeoDataFrame containing the data about provider-to-population ratio for each hospital.
    
    population_grid_gdf : str
    
        A GeoDataFrame containing the data about the number of inhabitants living in the Helsinki Region. The data should be 
        a Polygon grid with 250 meter resolution. 
    
    """
    
    # Remove those connections that were not reachable within given time
    filtered = travel_time_matrix.loc[~travel_time_matrix["travel_time"].isnull()]

    # Join the data to grid
    join = population_grid_gdf.merge(filtered, left_on="id", right_on="from_id")

    # Container for the results
    final_results = []

    # Group the data by origins and see how many hospitals are within reach
    catchments = join.groupby("from_id")

    for idx, catchment in catchments:

        # Merge the information with the hospitals
        hospitals_within = catchment.merge(ppr_gdf[["ppr", "facility_id", "name_en"]], left_on="to_id", right_on="facility_id")

        # What is the sum of PPRs for given pop grid code
        two_step_index = hospitals_within["ppr"].sum()

        # Fetch the geometry
        geom = catchment.iloc[0]["geometry"]

        # Update the data
        final_results.append({"geometry": geom, "accessibility": two_step_index})

    gdf = gpd.GeoDataFrame(final_results, geometry="geometry", crs="epsg:4326")
    return gdf

## Problem 0 - Download the data

Before starting the exercise, download the necessary data by executing the following cell (you only need to do this once): 

In [ ]:
# Download the data from a S3 bucket into 'data' folder
!wget -P data/ https://a3s.fi/swift/v1/AUTH_0914d8aff9684df589041a759b549fc2/Sustainability-GIS/Exercise-2-data.zip
    
# Extract the contents
!unzip -q data/Exercise-2-data.zip -d data/

If running the cell above does not work for some reason, you can [manually download the data](https://a3s.fi/swift/v1/AUTH_0914d8aff9684df589041a759b549fc2/Sustainability-GIS/Exercise-2-data.zip). If you do this, extract the contents of the Zip file (Exercise-2-data.zip) into the `<YOUR_FOLDER_CONTAINING_THIS_NOTEBOOK>/data` -folder.

## Problem 1 - Prepare network data for routing (1 point)

In this problem, we prepare a routable graph for our analysis using r5py and OpenStreetMap and GTFS data as input. You should:
 
- Use the `prepare_network()` function (above) to create the routable network. 
- As input for the function, you should specify the filepaths to the OSM and GTFS files which you downloaded in the Problem 0:
   - pass the OpenStreetMap file using `osm_fp` parameter, and 
   - the GTFS Zipfile using `gtfs_fp` parameter. 

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

## Problem 2 - First matrix: Calculate travel times from Otaniemi (2 points)

In this problem, the objective is to **calculate travel times from Otakaari 4 by public transport to all populated grid cells** in Helsinki Region.
Before you start, we recommend you to check the [Tutorial 2.2 from the course materials](https://sustainability-gis.readthedocs.io/en/latest/lessons/L2/r5py_calculating_travel_time_matrices.html) that demonstrates all the basic steps needed to solve this problem.

**Task 2.1. Prepare data.** In this task, you should:

1. Read the population grid data (250 m resolution) and convert it into a Point dataset called `pop_centroids`. In detail:
   1. Read the `population_grid_hsy.geojson` file into a variable `pop` which represents the population in the Helsinki Region as gridded Polygon data (you should have the dataset in the `data` directory)
   2. Create a copy from the `pop` dataset and store it into a varible `pop_centroids` (**Hint:** you can use `.copy()` function to create a copy of any GeoDataFrame or DataFrame)
   3. **Using the `pop_centroids` dataset** (i.e. the copy you just made): Calculate the centroids of the Polygons and store them into the column `geometry`. Notice, this will overwrite the original Polygon geometries. 
      - **Note:** Geopandas might warn you about 'centroid' not being correct. In our case, this does not matter and you can ignore this warning.
   
2. Using the `osmnx` library geocode a given address and ensure it is a Point dataset. In detail: 

   1. Geocode the address `"Otakaari 4, Espoo"` and store it into a variable called `otaniemi`.
   2. Using the `otaniemi` GeoDataFrame: Parse the values from the `index` and store them into a new column called `id` (**Hint:** you can use `.index` to get index values of a GeoDataFrame)
   3. Calculate the centroid of the Polygon (similarly as in previous step) and store it into the column `geometry`. Notice, this will overwrite the original Polygon geometries.
      - **Note:** Geopandas might warn you about 'centroid' not being correct. In our case, this does not matter and you can ignore this warning.
      
Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
import osmnx as ox
import geopandas as gpd

# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

**Task 2.2. Calculate travel times.** In this task, you should:

1. Initialize the `TravelTimeMatrixComputer` into a variable called `travel_time_matrix_computer` in a similar manner as shown in the [Tutorial 2.2](https://sustainability-gis.readthedocs.io/en/latest/lessons/L2/r5py_calculating_travel_time_matrices.html). As parameters, you should have following:
   - For the `transport_network` parameter, you should pass the routable `network` which you created in Problem 1.
   - For the `origins` parameter which indicates the location where the travel begins, you should pass the `otaniemi` GeoDataFrame.
   - For the `destinations` parameter which indicates all the destination locations, you should pass the `pop_centroids` GeoDataFrame.
   - For the `departure` parameter which indicates the time of departure, you should pass the time as a [datetime object](https://pythongis.org/part1/chapter-03/nb/03-temporal-data.html#constructing-datetime-objects) representing 7:30 in the morning on January 19th 2023 (see tutorial for example) 
   - For the `transport_modes` parameter which specifies the mode of travel or their combinations, you should define that the travel happens with a combination of `TRANSIT` and `WALK` (see tutorial for example). 
    
2. Calculate the travel times and store them into a variable `ttm`. 
   - You can do this by executing the `.compute_travel_times()` method of the `travel_time_matrix_computer` which you initialized earlier.

3. Make a table join between the Polygon grid dataset (variable `pop`) and the `ttm` DataFrame from the previous step. This allows us to map our results. 
   - You can do this by using the `pop.merge()` method and use the `id` column in the `pop` grid as the left key, and the `to_id` column in `ttm` as the right-key. 
   - **Hint:** Read more details about how table joins work from [Pythongis.org website](https://pythongis.org/part1/chapter-03/nb/01-data-manipulation.html#table-joins-combining-dataframes-based-on-a-common-key)

4. Visualize the travel times from Otaniemi using the `.explore()` function. 
   - You should use the `travel_time` as the `column` which will be used to visualize the data
   - You can pass `cmap="RdYlBu"` as the colormap if you want (also other colormaps are fine)
   - You can use `scheme="natural_breaks"` to classify the travel times according Natural Breaks classifier
   - You can use `k=8` to specify that you want to classify the travel times into 8 different classes
   - You can also plot the `otaniemi` marker on top of your map if you want (OPTIONAL)

As a result, you should have something like below (not necessarily exactly identical):
![Travel times from Otaniemi](img/travel_times_from_Otaniemi.png)


Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
from r5py import TravelTimeMatrixComputer, TransitMode, LegMode
import datetime

# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

## Problem 3 - Explore the dataset representing the simulated distribution of doctors in Helsinki Region (2 points)

**Task 3.1.** Explore healthcare facility data and find out how the doctors are distributed to different hospitals and health stations (based on simulated data). In this task, you should:
 
1. Read **hospital data** to a variable called `hospitals` (also in `data` directory) including information about the number of physicians at a given health care facility (hospital_data.geojson). 
  
2. Make a bubble map using the `make_bubble_map()` function provided for you in the Helper functions. The map should show the number of doctors per hospital or healthcare center in the Helsinki Region. The radius of the bubble should be based on the `physician_cnt` column. The `column` determining the color for the bubbles should be based on the `facility_type` column in the data.

As a result, you should have something like below (not necessarily exactly identical):
![Number of doctors in Helsinki Region](img/Hospitals_Bubble_map_Helsinki_Region.png)

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

**Task 3.2.** How many doctors are there? Explore the `hospitals` dataset and find out using programming:

1. How many doctors in total there are working at **Health Stations**? (print the result to the screen)
   - You should print something like following to the screen: `Doctors working at Health Stations: 677`.
2. How many doctors in total there are working at **University hospitals**? (print the result to the screen)
3. How many doctors in total there are working at **City hospitals**? (print the result to the screen)
   
**Hint:** You can find out answers for these by selecting the data based on `facility_type` column and then summing the number of doctors for these subsets which is stored in the `physician_cnt` column (separately for each facility type). 

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

## Problem 4 - Calculate 30 minute catchment areas by public transport for hospitals (2 points)

In this problem, we do the **first step** of "two-step floating catchment area" method, i.e. we aim to **i)** find out how many people are living within a given travel distance threshold (30 minutes travel distance by transit) from each hospital and **ii)** calculate the provider-to-population ratio (PPR) which gives an indication of the level of service, i.e. how many doctors per inhabitant there are within given catchment. 

**Task 4.1.** In this task, you should: 

1. Calculate the travel times from all healthcare facilities (i.e. the data stored in `hospitals` GeoDataFrame) to `pop_centroids`.
   - Follow the same approach as in **Problem 2, Task 2.2** and calculate the travel times by public transport+walking using the following departure time: 7:30 am on January 19th, 2023. The `origins` should be hospitals and the `destinations` should be `pop_centroids`. 
   - **Important**: As we want to calculate the 30-minute catchment areas, you need to specify the maximum travel time from the origin by using `max_time` parameter. You can specify this cut-off value by adding following argument to the TravelTimeMatrixComputer: `max_time=datetime.timedelta(seconds=30*60)`
   - Store the results in a variable called `times_from_hospital`

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

**Task 4.2.** Next we want to map the "dominance areas" for each hospital, i.e. we link the id of closest hospital to the population centroids. In this task, you should:

1. Determine for each destination (i.e. population centroid) which hospital is the closest one by using the `parse_id_of_the_closest_facility()` function provided for you in the Helper -functions. 
   - You should pass the result from previous step (i.e. `times_from_hospital` GeoDataFrame) and store the result into variable called `closest_facility`
   
2. Make a table join between the Polygon grid dataset (variable `pop`) and the `closest_facility` DataFrame from the previous step and store the data into variable `dominance_areas`. This allows us to map our results. This step is similar to what you did earlier in Task 2.2.
   - You can do this by using the `pop.merge()` method and use the `id` column in the `pop` grid as the left key, and the `to_id` column in `closest_facility` as the right-key. 
   - **Hint:** Read more details about how table joins work from [Pythongis.org website](https://pythongis.org/part1/chapter-03/nb/01-data-manipulation.html#table-joins-combining-dataframes-based-on-a-common-key)
   
3. Visualize the `dominance_areas` using the `.explore()` function. 
   - You should use the `facility_id` as the `column` which will be used to visualize the data
   - You can pass `cmap="jet"` as the colormap if you want (also other colormaps are fine)
   - You can also plot the `hospitals` markers on top of your map if you want (OPTIONAL)

As a result, you should have something like below (not necessarily exactly identical):
![Dominance_areas](img/catchment_areas_for_all_hospitals_in_HMA.png)   

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

## Problem 5: Calculate provider to population ratio (1 point)

**Task 5.1.** How many inhabitants are living under the dominance area of each healthcare facility? In this task, you should:

1. Group the `dominance_areas` GeoDataFrame based on `facility_id` and,
2. for each group, you should calculate the sum of population living under the given facility/group based on the `pop_cnt` column. 
3. Store the result in a DataFrame called `pop_catchments`

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

**Task 5.2.** Merge the population catchment information with the `hospitals` and calculate the provider-to-population ratio index. In this task, you should:

1. Make a table join between the `hospitals` GeoDataFrame and the `pop_catchments`, i.e. the data which you created in the earlier step, and store the result into a new variable called `ppr_hospitals`. You can do this by using `hospitals.merge()` method:
   1. the right DataFrame should be `pop_catchments`
   2. the left key for making the join should be `"id"`
   3. the right key for making the join should be `"facility_id"`

2. Calculate the provider-to-population ratio by dividing the number of doctors working at a given hospital with the number of potential patients, i.e. the population count that we calculated in previous task. You can do this by:
   1. using the formula: `number_of_doctors / population_count`
   2. `physician_cnt` column contains information about the number of doctors
   3. `pop_cnt` column contains information about the population count.
   
3. Answer to following questions by using programming to print out the answers to the screen:

   1. What is the healtcare facility with highest ppr?
   2. What is the healtcare facility with lowest ppr?
   
Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

## Problem 6: Calculate 2-step Floating Catchment Area index (1 point)

After you have done the first step of 2SFCA, we can finish the analysis by doing the second step in the analysis:

**Task 6.1** Calculate catchment areas from population centers. In this task, you should:

1. Calculate the travel times from all population centers (`pop_centroids`) to healthcare facilities (i.e. the data stored in `hospitals` GeoDataFrame). Note: It will take a bit longer to calculate these. 
   - Follow the same approach as in **Problem 4, Task 4.1** and calculate the travel times by public transport+walking using the following departure time: 7:30 am on January 19th, 2023. The `origins` should be population centroids and the `destinations` should be the hospitals. 
   - **Important**: As we want to calculate the 30-minute catchment areas, you need to specify the maximum travel time from the origin by using `max_time` parameter. You can specify this cut-off value by adding following argument to the TravelTimeMatrixComputer: `max_time=datetime.timedelta(seconds=30*60)`
   - Store the results in a variable called `times_from_population_centers`   
   
Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

**Task 6.2.** Calculate 2SFCA index and visualize the results. In this task, you should:

1. Calculate the 2SFCA index using the `calculate_2SFCA() `helper function which takes following datasets as input:
   1. `times_from_population_centers` GeoDataFrame representing the travel times, 
   2. `ppr_hospitals` representing the PPR values calculated for the hospitals, and 
   3. `pop` representing the population grid 

As a result, the function returns a GeoDataFrame with column `accessibility` containing the index based on 2SFCA method.
   
2. Plot the results using `.explore()` method:
   - You should use the `accessibility` as the `column` which will be used to visualize the data
   - You can pass `cmap="Blues"` as the colormap if you want (also other colormaps are fine)
   - You can use `scheme="quantiles"` to classify the values according Quantiles classifier which reveals well the accessibility patterns
   - You can use `k=4` to specify that you want to classify the travel times into four different classes
   - You can also plot the `hospitals` markers on top of your map if you want (OPTIONAL)

As a result, you should get something like following (not necessarily totally identical):
 
!["Healthcare accessibility map in Helsinki Region](img/hospital_accessibility_2SFCA_HelsinkiRegion.png)

Please write your solution to the cell below (remove the `raise NotImplementedError()` code). You can create new cells as well if needed.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

## Problem 7 - How long did it take? Optional feedback (1 point)

To help developing the exercises, and understanding the time that it took for you to finish the Exercise, please provide an estimate of how many hours you spent for doing this exercise? *__Hint:__ To "activate" this cell in Editing mode, double click this cell. If you want to get this cell back in the "Reading-mode", press Shift+Enter.*


I spent approximately this many hours: **X hours**

In addition, if you would like to give any feedback about the exercise (optional), please provide it below:

**My feedback:**